# How not to use Criterions

Let's perform an analysis on bitcoin data using different criteria:

1. Akaike's Information Criterion:
    $$ n \ln(\sigma_{z}^{2}) + 2 k$$
2. Bayesian Infromation Criterion:
    $$ n \ln(\sigma_{z}^{2}) + k\ln(n) $$
3. Final Prediction Error:
    $$\sigma_{z}^{2} \frac{N+M+1}{N-M-1}$$

In [1]:
# What we will need:
import warnings
warnings.filterwarnings('ignore')

import csv # for reading
import numpy as np # for math

import plotly                  # for
import plotly.graph_objs as go # plotting

from itertools import product # for clearer iterations
# local tools to read the time series
from libs.localfunctions import get_total_num_rows, get_bitcoint_rows
# for creating/evaluating models
# from frama_numpy_performance import frama_perf
from statsmodels.tsa.arima_model import AR, ARMA, ARIMA
import scipy

plotly.offline.init_notebook_mode(connected=True)

In [3]:
def get_model_criterions(model, errors):
    N = np.size(errors)
     
    num_params = len(model.arparams) + len(model.maparams)
    
#     sz2 = np.std(dst_array[d:] - model_output)
    sz2 = np.std(errors) ** 2
    #Akaike Information Criterion
    aic = N  * np.log(sz2) + 2.0*num_params
    # Bayesian Information Criterion
    bic = N * np.log(sz2) + num_params * np.log(N)
    # Final Prediction Error
    fpe = sz2*(N + num_params + 1.0)/(N - num_params - 1.0)
    
    return aic, bic, fpe, sz2

## Bitcoin TimeSeries

In [4]:
# # # Read file with time series data
dataset_name = "data/" + "bitstampUSD_1-min_data_2012-01-01_to_2017-10-20.csv"
num_training_rows =  10000 # 100000
num_testing_rows = 5000

In [5]:
with open(dataset_name, 'r') as csvfile:
    csv_reader = csv.DictReader(csvfile, delimiter=',')
    keys, training_rows = get_bitcoint_rows(csv_reader, num_training_rows)
    _, testing_rows = get_bitcoint_rows(csv_reader, num_testing_rows)
    print(keys)


dict_keys(['Weighted_Price', 'Volume_(Currency)', 'Open', 'Timestamp', 'High', 'Volume_(BTC)', 'Close', 'Low'])


In [6]:
    index_Weighted_Price = list(keys).index('Weighted_Price')
    index_Timestamp = list(keys).index('Timestamp')
    print(index_Weighted_Price)
    print(index_Timestamp)
    X_train = training_rows[:,index_Weighted_Price]
    X_test = testing_rows[:,index_Weighted_Price]
    Y_train = X_train[1:] - X_train[:-1]
    Y_test = X_test[1:] - X_test[:-1]
    Timestamp_train = training_rows[:, index_Timestamp]
    Timestamp_test = testing_rows[:, index_Timestamp]

0
3


In [7]:
# # # plot the total bitcoin Series
trace = go.Scatter(
    x = Timestamp_train,
    y = X_train,
    mode = 'lines',
    name = 'bitcoin time-series per minute')

layout = dict(title = 'Bitcoin time series',
              xaxis = dict(title = 'Timestamp'),
              yaxis = dict(title = 'Weighted Price'))

fig = dict(data=[trace], layout=layout)
plotly.offline.iplot(fig, filename='total-series-plot')

## criterions for AR processes

In [8]:
k_AR_max = 5
k_MA = 0
diff = 0
criterionsAR = np.zeros((k_AR_max+1, 4))
criterionsAR.fill(np.nan)
for k_AR in range(1, k_AR_max+1):
    try:
        model = ARIMA(Y_train, order=(k_AR, diff, k_MA))
        model_fit = model.fit(disp=0)
#         x_hat = model.predict(params=model_fit.params)
#         x_hat = np.cumsum(np.insert(model.predict(Y),0,X_train[0]))
        mse = np.std(model.geterrors(model_fit.params))

        print(k_AR,',', mse)
    except:
        print("avoiding model evaluation")
        criterionsAR[k_AR,:] = [np.nan, np.nan, np.nan, np,nan]
        continue

#     model_output = get_model_output(model_fit, testing_rows[:,0], diff)
    criterionsAR[k_AR,:] = get_model_criterions(model_fit, model.geterrors(model_fit.params))

1 , 0.009516125392
2 , 0.00951485647364
3 , 0.00951281491639
4 , 0.00951239003404
5 , 0.0095123772142


In [9]:
# # # plot criterions for AR processes using plot.ly

X = np.arange(k_AR_max)
# # # MSE
trace = go.Scatter(
    x = X,
    y = criterionsAR[:,3],
    mode = 'lines+markers')
layout = dict(title = 'MSE',
              xaxis = dict(title = '#AR parameters'),
              yaxis = dict(title = 'MSE'))
fig = dict(data=[trace], layout=layout)
plotly.offline.iplot(fig, filename='AR-MSE')

# # AIC
trace = go.Scatter(
    x = X,
    y = criterionsAR[:,0],
    mode = 'lines+markers')
layout = dict(title = 'AIC criterion for AR',
              xaxis = dict(title = '#AR parameters'),
              yaxis = dict(title = 'AIC error'))
fig = dict(data=[trace], layout=layout)
plotly.offline.iplot(fig, filename='AR-AIC')

# # BIC
trace = go.Scatter(
    x = X,
    y = criterionsAR[:,1],
    mode = 'lines+markers')
layout = dict(title = 'BIC criterion for AR',
              xaxis = dict(title = '#AR parameters'),
              yaxis = dict(title = 'BIC error'))
fig = dict(data=[trace], layout=layout)
plotly.offline.iplot(fig, filename='AR-BIC')

# # FPE
trace = go.Scatter(
    x = X,
    y = criterionsAR[:,2],
    mode = 'lines+markers')
layout = dict(title = 'FPE criterion for AR',
              xaxis = dict(title = '#AR parameters'),
              yaxis = dict(title = 'FPE error'))
fig = dict(data=[trace], layout=layout)
plotly.offline.iplot(fig, filename='AR-FPE')

In [10]:
# # # Best AR output prediction plot VS dataset plot
k_AR_best = np.zeros((3))
k_AR_best[0] = np.int(np.nanargmin(criterionsAR[:,0], axis=0)) # with AIC
k_AR_best[1] = np.int(np.nanargmin(criterionsAR[:,1], axis=0)) # with BIC
k_AR_best[2] = np.int(np.nanargmin(criterionsAR[:,2], axis=0)) # with FPE
print(k_AR_best)

[ 3.  1.  3.]


In [11]:
# # # plot optimal AR prediction in test sample
model = ARIMA(Y_train, order=(int(k_AR_best[2]), diff, k_MA))
model_fit = model.fit(disp=0)

model = ARIMA(Y_test, order=(int(k_AR_best[2]), diff, k_MA))
model.fit()
x_hat = np.cumsum(np.insert(model.predict(params=model_fit.params), 0, X_test[0]))

trace = go.Scatter(
    x = Timestamp_test,
    y = X_test,
    mode = 'lines+markers')

trace_hat = go.Scatter(
    x = Timestamp_test,
    y = x_hat,
    mode = 'lines+markers')

layout = dict(title = 'Testing AR result',
              xaxis = dict(title = 'Timestamp'),
              yaxis = dict(title = 'weighted-price'))
fig = dict(data=[trace,trace_hat], layout=layout)
plotly.offline.iplot(fig, filename='AR_test')


trace = go.Scatter(
    x = Timestamp_test,
    y = Y_test,
    mode = 'lines+markers')

trace_hat = go.Scatter(
    x = Timestamp_test,
    y = model.predict(params=model_fit.params),
    mode = 'lines+markers')

layout = dict(title = 'Testing AR result',
              xaxis = dict(title = 'Timestamp'),
              yaxis = dict(title = 'weighted-price'))
fig = dict(data=[trace,trace_hat], layout=layout)
plotly.offline.iplot(fig, filename='AR_test')

## criterions for ARIMA processes

In [12]:
k_MA_max = 1
k_AR_max = 5
diff = 1
criterionsARIMA = np.zeros((k_AR_max+1, k_MA_max+1, 4))
criterionsARIMA.fill(np.nan)


for k_AR, k_MA in product(range(1, k_AR_max+1), range(0, k_MA_max+1)): # loop in the set [0, k_AR_max] X [1, k_MA_max]
    print("evaluating process (AR,MA) = " + str((k_AR, k_MA)))
    try:
        model = ARIMA(Y_train, order=(k_AR, diff, k_MA))
        model_fit = model.fit(disp=1)
        x_hat = model.predict(params=model_fit.params)
        mse = np.std(model.geterrors(model_fit.params))
        print("(%d,%d), %f"%(k_AR, k_MA, mse))
    except ValueError:
        print("avoiding model evaluation")
        criterionsARIMA[k_AR,k_MA,:] = [np.nan, np.nan, np.nan, np.nan]
        continue
    criterionsARIMA[k_AR,k_MA,:] = get_model_criterions(model_fit, model.geterrors(model_fit.params))

evaluating process (AR,MA) = (1, 0)
(1,0), 0.011529
evaluating process (AR,MA) = (1, 1)
(1,1), 0.009518
evaluating process (AR,MA) = (2, 0)
(2,0), 0.010871
evaluating process (AR,MA) = (2, 1)
(2,1), 0.009516
evaluating process (AR,MA) = (3, 0)
(3,0), 0.010550
evaluating process (AR,MA) = (3, 1)
(3,1), 0.009521
evaluating process (AR,MA) = (4, 0)
(4,0), 0.010366
evaluating process (AR,MA) = (4, 1)
(4,1), 0.009516
evaluating process (AR,MA) = (5, 0)
(5,0), 0.010235
evaluating process (AR,MA) = (5, 1)
(5,1), 0.009529


In [13]:
# # # plot criterions for ARIMA processes using plot.ly
data = []
X = np.arange(k_MA_max)

# # AIC
for k_AR in range(1, k_AR_max):
    trace = go.Scatter(
        x = X,
        y = criterionsARIMA[k_AR,:,0],
        mode = 'lines+markers',
        name = 'AR=%d'%k_AR)
    data.append(trace)
layout = dict(title = 'AIC criterion for ARIMA',
              xaxis = dict(title = '#MA parameters'),
              yaxis = dict(title = 'AIC error'))

fig = dict(data=data, layout=layout)
plotly.offline.iplot(fig, filename='ARIMA-AIC')

# # BIC
data = []
for k_AR in range(1, k_AR_max):
    trace = go.Scatter(
        x = X,
        y = criterionsARIMA[k_AR,:,1],
        mode = 'lines+markers',
        name = 'AR=%d'%k_AR)
    data.append(trace)
layout = dict(title = 'BIC criterion for ARIMA',
              xaxis = dict(title = '#MA parameters'),
              yaxis = dict(title = 'BIC error'))

fig = dict(data=data, layout=layout)
plotly.offline.iplot(fig, filename='ARIMA-BIC')

# # FPE
data = []
for k_AR in range(1, k_AR_max):
    trace = go.Scatter(
        x = X,
        y = criterionsARIMA[k_AR,:,2],
        mode = 'lines+markers',
        name = 'AR=%d'%k_AR)
    data.append(trace)
layout = dict(title = 'FPE criterion for ARIMA',
              xaxis = dict(title = '#MA parameters'),
              yaxis = dict(title = 'FPE error'))

fig = dict(data=data, layout=layout)
plotly.offline.iplot(fig, filename='ARIMA-FPE')

In [14]:
k_ARIMA_best = np.zeros((3))
k_ARIMA_best[0] = np.int(np.nanargmin(criterionsARIMA[:,:,0])) # with AIC
k_ARIMA_best[1] = np.int(np.nanargmin(criterionsARIMA[:,:,1])) # with BIC
k_ARIMA_best[2] = np.int(np.nanargmin(criterionsARIMA[:,:,2])) # with FPE
print(k_ARIMA_best)
print(criterionsARIMA[:,:,2])

[ 5.  3.  5.]
[[             nan              nan]
 [  1.32977314e-04   9.06390492e-05]
 [  1.18259013e-04   9.06346482e-05]
 [  1.11384979e-04   9.07314635e-05]
 [  1.07571000e-04   9.06607110e-05]
 [  1.04881631e-04   9.09378596e-05]]


In [15]:
# # # plot optimal ARIMA prediction
k_AR = 3
diff = 0
k_MA = 0
model = ARIMA(Y_train, order=(k_AR, diff, k_MA))
model_fit = model.fit(disp=0)

model = ARIMA(Y_test, order=(int(k_AR), diff, k_MA))
model.fit()
x_hat = np.cumsum(np.insert(model.predict(params=model_fit.params), 0, X_test[0]))
trace = go.Scatter(
    x = Timestamp_test,
    y = X_test,
    mode = 'lines+markers')

trace_hat = go.Scatter(
    x = Timestamp_test,
    y = x_hat,
    mode = 'lines+markers')

layout = dict(title = 'Testing ARIMA(%d,%d,%d) result'%(k_AR,1,k_MA),
              xaxis = dict(title = 'Timestamp'),
              yaxis = dict(title = 'weighted-price'))
fig = dict(data=[trace,trace_hat], layout=layout)
plotly.offline.iplot(fig, filename='ARIMA_test')


## More Problems

These criterions are the basics and cannot help in more complicated problems, like desiding the architecture of an LSTM layer. 
1.  No computational complexity descriptor
![LSTM_simple](LSTM3-SimpleRNN.png "Simple LSTM layer")
![LSTM_full](LSTM3-chain.png "Full LSTM layer")


2\.  Many parameters, lead to many plots

3\.  Many non-linear cases require exhaustive search
![brute_force](bruteForceNQueens04.png "Brute Force")